<a href="https://colab.research.google.com/github/sakuronohana/cas_datenanalyse/blob/master/Semesterarbeit/CAS_DA_SA_Payment_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img 
src="https://www.ffhs.ch/templates/ffhs/img/logo@2x.png" width="100"> 
###DaAn, Data Analysis, MAS/CAS Web4B 2018, ZH1, FS19, Dr. Tödtli Beat###

##*Semesterarbeit von Patrik Di Lena*##
#Betrugserkennung Zahlungsverkehr#



##Ziel##
Auf der Grundlage von maschinellen Lernalgorithmen soll ein Betrugserkennungssystem für den Zahlungsverkehr einer ausschliesslich in der Schweiz tätigen Bank erstellt werden. Auf Basis historisierter Daten in Bezug auf das E-Banking Session- und Zahlungsverhalten von Bankkunden sollen mögliche Betrugsintensionen erkannt und für bestimmte Risikogruppen vorhergesagt werden.

##Aufgabenstellung##

Aus dem oben formulierten Ziel ergeben sich zwei wesentliche Aufgabenstellungen:
1.	Erkennung betrügerischer Handlungen im Zahlungsverkehr
2.	Ermittlung potenzieller Kundenrisikogruppen

##Betrugsanfällige Risikogruppen##
Die für den Betrug im Zahlungsverkehr anfälligen Gruppen werden auf Basis der folgenden Merkmale durch den Lernalgorithmus bestimmt:

*	Geschlecht
*	Alter
*	Zivilstand
*	Nationalität
*	Kontosaldo
*	Wohnort
*	Region
*	Zahlungsmethode (E-Rechnung, Einzahlungsschein etc.)
*	Rechtsform (Juristisch oder natürliche Person)

## Datensatz##
Die im Rahmen dieser Semesterarbeit verwendeten Daten entstammen, von ihrer Datenstruktur her, aus einem bereits eingesetzten nicht ML-basierten E-Banking Betrugserkennungssystem. Die Datenwerte wurde zu Wahrung der datenschutzrechtlichen und bankengesetzlichen Aspekte mit Hilfe des folgenden Python-Scripts künstlich erzeugt und beinhalten somit keine reale Zahlungs-, Kunden-, und betrugsspezifischen Informationen. Weiter wurden einige durch die Bank definierten Spezifikationen bzw. Merkmale aus sicherheitstechnischen Gründen umbenannt.  

Datenselektion:

- 5'000 synthetische Kunden (Vorname, Name, Strasse, Ort, Kanton, Sprachregion, Geschlecht, Alter, Zivilstand,  Nationalität, Kontonummer, Vertragsnummer, Rechtsform )

- 100'000 synthetische Zahlungen (Transaktionsid, ,Erstellungdatum/Zeit, Empfängerkonto, Empfängerbank, Empfängerland, Währung, Betrag, Valuta Datum)








###Datengeneratoren###

Während die Stammdaten hauptsächlich im Excel manuell von verschiedenen Quellen 

Datenquellen:

Open Government Data https://opendata.swiss/de/


###IBAN Generator###

In [107]:
# IBAN Generator

import datetime
import numpy as np
import pandas as pd

# Datenquellen Github
ibandb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/iban_list.csv'

# IBAN Generator in elektronischer Form

iban_cc = pd.read_csv(ibandb_url, delimiter=';')
iban_cc.columns=['Country','Code','Checkdig','Length','Sample','Bank']

# IBAN Gen. 1.Teil - Erstellung IBAN Länder- und Bankencode
code = np.random.choice(np.ravel(iban_cc['Code']).astype(str))
checkdig = np.random.choice(np.ravel(iban_cc['Checkdig']).astype(str))
iban_country_code = code + checkdig
iban_bank_code = np.random.choice((iban_cc.loc[iban_cc['Code'] == code])['Length']).astype(str)
iban_ccbc = iban_country_code + "00" + iban_bank_code 

# IBAN Gen. 2.Teil - Erstellung Kontonummer und Zusammensetzung der IBAN
eiban = []
for row in range(20):
    part_eiban = pd.DataFrame(np.int64(np.random.randint(2000000000000,3000000000000,20800)))
    eiban = iban_ccbc + part_eiban.astype(str)

# Zufällige Auswahl einer IBAN aus dem generierten Datensatz
eiban = np.ravel(eiban)
eiban = np.random.choice(eiban)
print (eiban)





LC7200322800994985921


###Payment Generator###

In [262]:
# Erstellung von Transaktionsdaten 
# Daten Payment ID z.B. 1059295797

from datetime import datetime, timedelta
import random
import numpy as np
import pandas as pd

# Datenquellen Github
ibandb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/iban_list.csv'

# IBAN Generator in elektronischer Form

iban_cc = pd.read_csv(ibandb_url, delimiter=';')
iban_cc.columns=['Country','Code','Checkdig','Length','IBAN','Bank']

# IBAN Gen. 1.Teil - Erstellung IBAN Länder- und Bankencode
code = np.random.choice(np.ravel(iban_cc['Code']).astype(str))
checkdig = np.random.choice(np.ravel(iban_cc['Checkdig']).astype(str))
iban_country_code = code + checkdig
iban_bank_code = np.random.choice((iban_cc.loc[iban_cc['Code'] == code])['Length']).astype(str)
iban_ccbc = iban_country_code + "00" + iban_bank_code 


# IBAN Gen. 2.Teil - Erstellung Kontonummer und Zusammensetzung der IBAN
eiban = []
for row in range(20):
    part_eiban = pd.DataFrame(np.int64(np.random.randint(2000000000000,3000000000000,20800)))
    eiban = iban_ccbc + part_eiban.astype(str)

# Zufällige Auswahl einer IBAN aus dem generierten Datensatz
eiban = np.ravel(eiban)
eiban = np.random.choice(eiban)


# Datenquellen Github
custdb_url = 'https://raw.githubusercontent.com/sakuronohana/cas_datenanalyse/master/Semesterarbeit/Dataset/cutomer_base_data.csv'

cust_data = pd.read_csv(custdb_url, delimiter=';')
cust_data.columns=['anrede','vorname','nachname','strasse','nr','plz','ort',
                   'kanton','sprachregion',	'nation',	'sexcode','mobile_nr',	
                   'vertrag_nr','konto_nr','auth_device']

# Importierte Kundendaten bereinigen
cust_data = cust_data.fillna(0)
cust_data_dtype = {'plz':int,'mobile_nr':int,'vertrag_nr':int}
cust_data = cust_data.astype(cust_data_dtype)

# Zufallsauswahl Kundendaten
cd_contract = np.random.choice(np.ravel(cust_data['vertrag_nr']))
cd_kontonr = cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['konto_nr']]
#get_cids = cust_data.loc[cust_data['vertrag_nr'] == cd_contract,['konto_nr','vertrag_nr',
                                                                 #'nachname','vorname','sexcode',
                                                                 #'plz','ort','kanton',
                                                                #'sprachregion','auth_device']]

# Zufallsauswahl Empfängerbank
bc_bank = np.random.choice(np.ravel(iban_cc['Bank']))
bc_iban = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['IBAN']])
print(bc_iban)



# Erstellung der Zahlungs-Basisinformationen 

start_date = datetime(2018,1,1,8,0,0)
trx_data = pd.DataFrame(columns=['timestamp','paym_id','send_bc_nr','amount',
                                 'rcv_bc_nr','rcv_bc_iban','rcv_bc_country','rcv_bc_code',
                                 'vertrag_nr'])

for i in range(10):
    trx_data.loc[i,['timestamp']] = pd.Timestamp(datetime(2018,1,1,8,0,0)+timedelta(minutes=30*i)).strftime("%Y.%m.%d %H:%M")
    trx_data.loc[i,['paym_id']] =  np.int64(np.random.randint(1000000000000,2000000000000,1))
    trx_data.loc[i,['send_bc_nr']] =  'CHBANK71XXX'
    trx_data.loc[i,['amount']] = np.int64(np.random.randint(100,100000,1)).astype(str)
    trx_data.loc[i,['rcv_bc_nr']] = bc_bank = np.random.choice(np.ravel(iban_cc['Bank']).astype(str))
    trx_data.loc[i,['rcv_bc_iban']] = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['IBAN']])
    trx_data.loc[i,['rcv_bc_country']] = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['Country']])
    trx_data.loc[i,['rcv_bc_code']] = np.ravel(iban_cc.loc[iban_cc['Bank'] == bc_bank,['Code']])
    trx_data.loc[i,['vertrag_nr']] = cd_contract = np.random.choice(np.ravel(cust_data['vertrag_nr']))

    

print(trx_data)




['HU02116000060000000064247067']
          timestamp        paym_id   send_bc_nr amount    rcv_bc_nr  \
0  2018.01.01 08:00  1780704164168  CHBANK71XXX  92003  CRBANK20XXX   
1  2018.01.01 08:30  1763315893851  CHBANK71XXX  59062  LVBANK46XXX   
2  2018.01.01 09:00  1623169958314  CHBANK71XXX  66303  BGBANK19XXX   
3  2018.01.01 09:30  1065551787713  CHBANK71XXX  20152  DKBANK21XXX   
4  2018.01.01 10:00  1841613114680  CHBANK71XXX  58419  IEBANK36XXX   
5  2018.01.01 10:30  1553912925650  CHBANK71XXX   3711  PKBANK61XXX   
6  2018.01.01 11:00  1424863228642  CHBANK71XXX  82879  MCBANK56XXX   
7  2018.01.01 11:30  1633221052681  CHBANK71XXX  14417  IEBANK36XXX   
8  2018.01.01 12:00  1587280910145  CHBANK71XXX  85795  DKBANK21XXX   
9  2018.01.01 12:30  1228589550254  CHBANK71XXX  96888  GEBANK29XXX   

                   rcv_bc_iban rcv_bc_country rcv_bc_code vertrag_nr  
0       CR79015202220005614288     Costa Rica          CR      66547  
1        LV02HABA0551007820897       Lettla

In [149]:


import pandas as pd
from random import randint

df = pd.DataFrame(columns=['A','B','C','D','E'])

for i in range(5): #add 5 rows of data
    df.loc[i, ['A']] = randint(0,99)
    df.loc[i, ['B']] = randint(0,99)
    df.loc[i, ['C']] = randint(0,99)
    df.loc[i, ['D']] = randint(0,99)
    df.loc[i, ['E']] = randint(0,99)
    
df

,A,B,C,D,E
0,82.0,93.0,1.0,5.0,8.0
1,4.0,34.0,45.0,13.0,88.0
2,87.0,41.0,54.0,41.0,43.0
3,81.0,44.0,98.0,3.0,83.0
4,47.0,52.0,20.0,45.0,60.0
